# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [10]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key
gmaps.configure(api_key = g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [27]:
output_data_file = "../output_data/cities.csv"

df = pd.read_csv(output_data_file)
df1 = df.drop(columns=["Unnamed: 0"])
df2 = df1.drop(columns=["City_ID.1"])
df2
df3 = df2.drop(columns=["City_ID.1.1"])
df3
vacation_data_df = df3.drop(columns=["City_ID"])
vacation_data_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,jamestown,42.10,-79.24,28.40,68,75,23.04,US,1587519044
1,kambove,-10.88,26.60,56.88,64,92,4.34,CD,1587519044
2,nanortalik,60.14,-45.24,37.36,79,54,15.39,GL,1587519044
3,lebu,-37.62,-73.65,54.46,61,100,4.88,CL,1587519135
4,busselton,-33.65,115.33,70.00,68,100,17.13,AU,1587519100
...,...,...,...,...,...,...,...,...,...
562,verkhnevilyuysk,63.45,120.32,39.16,71,32,4.63,RU,1587519419
563,uchiza,-8.46,-76.46,72.46,57,24,2.59,PE,1587519419
564,samana,19.21,-69.34,82.40,74,20,11.41,DO,1587519204
565,sabang,5.89,95.32,85.59,73,100,8.90,ID,1587519419


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [31]:
locations = vacation_data_df[["Lat", "Lng"]]
humidity = vacation_data_df["Humidity"].astype(float)

fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)
fig.add_layer(heat_layer)
fig




Figure(layout=FigureLayout(height='420px'))

In [40]:
new_df = pd.DataFrame(vacation_data_df["City"]["Max Temp"]["Wind Speed"],["Cloudiness"])
new_df


KeyError: 'Wind Speed'

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
hotel_df = 

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
